MÔ TẢ

Bài tập tuần 2: ETL load file excel vào trong db
File gồm có 3 sheet:
load vào trong database nhé

khởi tạo thư viện cần thiết

In [ ]:
import pandas as pd
import yaml
import pyodbc
from sqlalchemy import create_engine
from prefect import flow, task, get_run_logger
from prefect.tasks import task_input_hash
from datetime import timedelta

import sys
sys.path.append(r"G:/My Drive/HỌC TẬP DUE/NNT_năm 4 kì 1/DW & DM/config")
import mdl_db_connection

In [ ]:
# C:\Users\admin\anaconda3\pkgs\graphviz-2.50.0-hdb8b0d4_0\Library\include\graphviz
# C:\Users\admin\anaconda3\Lib\site-packages\sphinx\templates\graphviz
# C:\Users\admin\anaconda3\pkgs\sphinx-4.4.0-pyhd3eb1b0_0\site-packages\sphinx\templates\graphviz
# C:\Users\admin\anaconda3\Lib\site-packages\graphviz
# C:\Users\admin\anaconda3\Library\include\graphviz
# C:\Users\admin\anaconda3\pkgs\graphviz-2.50.0-hdb8b0d4_0\Library\bin
# "C:\Users\admin\anaconda3\pkgs\graphviz-2.50.0-hdb8b0d4_0\Library\bin"

## ETL

### 2.1 extract data

In [ ]:
@task(name="convert sheet to dataframe", 
      description="do file excel có 3 sheet nên mình lấy luôn 3 sheet từ 1 file này và chuyển sang df",
      task_run_name="convert sheet to dataframe",
      cache_key_fn=task_input_hash, 
      cache_expiration=timedelta(days=1))
def extract():
    excel_file_path = r"G:\My Drive\HỌC TẬP DUE\NNT_năm 4 kì 1\DW & DM\ETL_Sale week 2\ETL SALE PYTHON\SaleDataset.xlsx"
    logger = get_run_logger()
    logger.info("convert sheet ORDER to df")
    df_ORDER = pd.read_excel(excel_file_path, sheet_name="ORDER")

    logger.info("convert sheet CUSTOMER to df")
    df_CUSTOMER = pd.read_excel(excel_file_path, sheet_name="CUSTOMER")

    logger.info("convert sheet PRODUCT to df")
    df_PRODUCT = pd.read_excel(excel_file_path, sheet_name="PRODUCT")
    return df_ORDER, df_CUSTOMER, df_PRODUCT



### 2.2 transform

In [ ]:
@task(
        name="merge join df & add column", 
        description="yêu cầu là nối lại với nhau để tạo bảng lớn và thêm cột profit",
        task_run_name="merge joni df & add column",
        cache_key_fn=task_input_hash, 
        cache_expiration=timedelta(days=1)
    )
def transform_df(df_ORDER, df_CUSTOMER, df_PRODUCT):
    logger = get_run_logger()
    logger.info("Nối 2 bảng ORDER và Product, key ProductID")
    df_order_product = pd.merge(df_ORDER, df_PRODUCT, on='ProductID', how='outer')

    logger.info("Nối 2 bảng ORDER_PRODUCT và CUSTOMER, key CustomerId")
    df_OPC = pd.merge(df_order_product, df_CUSTOMER, on='CustomerId', how='outer')

    logger.info("Tạo thêm cột profit")
    df_OPC['Profit'] = df_OPC['Sales'] - df_OPC['COGS']
    return df_OPC

### 2.3 load dáta

In [ ]:
@task(name="load df vào database", 
      description="lưu vào db để sau sử dụng lại", 
      task_run_name="load df vào databasei",
      cache_key_fn=task_input_hash, 
      cache_expiration=timedelta(days=1)
    )
def load(df_OPC):
    logger = get_run_logger()
    logger.info("Kết nối đến db")
    conn_str, engine = mdl_db_connection.connect_db()
    conx = pyodbc.connect(conn_str) 

    # bảng muốn lưu trữ dữ liệu
    table_name = 'ETL_SaleDataset_week2'
    

    #kiểm tra
    try:
      # Lưu DataFrame vào cơ sở dữ liệu
      df_OPC.to_sql(table_name, con=engine, if_exists='replace', index=False)
      conx.commit()
      logger.info(f"Đã tạo thành công bảng {table_name}")
    except Exception as e:
      print(f"Bị lỗi: {str(e)}")
    finally:
      # Đóng kết nối pyodbc
      conx.close()

    

## Quản lý flow

In [ ]:
@flow(
      name = "FLOW ETL EXCEL TO DB - WEEK 2 - USING PREFECT PYTHON"
      , flow_run_name="ETL excel to db"
      , retries=1
      , retry_delay_seconds=5
      , log_prints=True
      , description="Chạy hết kết hợp cả lấy dữ liệu từ 3 sheet trong 1 file excel, sau đó load vào pandas và xử lý cuối cùng đẩy về db"
      )
def build_flow():
    logger = get_run_logger()
    logger.info("Thực hiện extract dữ liệu")

    df_ORDER, df_CUSTOMER, df_PRODUCT = extract()


    logger.info("Thực hiện transform dữ liệu")
    df_OPC = transform_df(df_ORDER, df_CUSTOMER, df_PRODUCT)



    logger.info("Load dữ liệu vào database")
    df = load(df_OPC)

    return flow

In [ ]:
if __name__ == "__main__":
    flow = build_flow() 

# flow = build_flow() 


test tính năng

khởi tạo

In [ ]:
# import sys
# !{sys.executable} -m pip install -U ydata-profiling[notebook]
# !jupyter nbextension enable --py widgetsnbextension

In [34]:

from ydata_profiling import ProfileReport

excel_file_path = r"G:\My Drive\HỌC TẬP DUE\NNT_năm 4 kì 1\DW & DM\ETL_Sale week 2\ETL SALE PYTHON\SaleDataset.xlsx"
df_ORDER = pd.read_excel(excel_file_path, sheet_name="ORDER")

profile = ProfileReport(df_ORDER, title="Pandas Profiling Report")
profile.to_widgets()
# profile.to_notebook_iframe()



Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]